# 천안 양산 사업장 수요예측 모델 개선 

**due date** : 3/22<p>
**goal** : 미래 24개월 장납기 수입 소자 필요량 예측을 위한 Base model 생성<p>
**data** : INV_Transaction_Summary 
+ x1 = 해당 월의 수량
+ PO_RECEIPT : 재고수불

**참고사항**   
+ 대체품 반영 X (-> 과수량 예측 Issue)
+ 같은 월에 아이템 코드 중복 있는 경우 merge (-> 추후 필요에 따라 분류)
+ 아이템 코드 맨앞 0 누락 주의
+ desc 맨 앞 = 제품 타입
+ 빈도등급에 따른 처리 방안 (-> 평균 vs 빈도수 ...)


## setting

### library & data import

In [1]:
import os
import pandas as pd
import numpy as np
import datetime 
from prophet import Prophet 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
inv = pd.read_excel('../data/INV_Transacton_Summary.xlsx', engine = 'openpyxl', sheet_name = 0)
df_inv = inv.copy()

### data overview

In [3]:
df_inv.head()

,Unnamed: 0,YYYYMM,IEM_CODE,ITEM_DESCRIPTION,PO_RECEIPT,SPG,UOM,OTHER_ISSUE,OTHER_RECEIPT,SALES_ISSUE,WIP_ISSUE,WIP_RECEIPT
0,0,201401,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",3528.0,500,ea,1980.0,1960.0,1960.0,160.0,0.0
1,1,201402,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",1568.0,500,ea,1862.0,1862.0,1862.0,0.0,0.0
2,2,201403,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",2744.0,500,ea,1568.0,1568.0,1568.0,0.0,0.0
3,3,201404,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",2450.0,500,ea,2450.0,2450.0,2450.0,96.0,0.0
4,4,201405,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",392.0,500,ea,2254.0,2254.0,2254.0,20.0,0.0


In [47]:
df_inv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177961 entries, 0 to 177960
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0                     177961 non-null  int64  
 1   YYYYMM            177961 non-null  int64  
 2   SPG               177961 non-null  int64  
 3   IEM_CODE          177961 non-null  object 
 4   ITEM_DESCRIPTION  177961 non-null  object 
 5   UOM               177961 non-null  object 
 6   SUBINV            177961 non-null  object 
 7   PO_RECEIPT        177961 non-null  int64  
 8   WIP_RECEIPT       177961 non-null  int64  
 9   OTHER_RECEIPT     177961 non-null  int64  
 10  WIP_ISSUE         177961 non-null  float64
 11  SALES_ISSUE       177961 non-null  int64  
 12  OTHER_ISSUE       177961 non-null  float64
dtypes: float64(2), int64(7), object(4)
memory usage: 17.7+ MB


In [3]:
df_inv = df_inv.drop(df_inv.columns[0], axis = 1)
df_inv = df_inv.astype({'YYYYMM':'str'})
df_inv['ITEM_TYPE'] = df_inv['ITEM_DESCRIPTION'].str.split(',').str[0]

In [4]:
num_order = len(df_inv)
num_code = len(set(df_inv['IEM_CODE']))
num_item = len(set(df_inv['ITEM_DESCRIPTION']))
num_type = len(set(df_inv['ITEM_TYPE']))

print(f'주문 정보 총량 : {num_order}')
print(f'고유 아이템 code : {num_code}')
print(f'고유 아이템 desc : {num_item}')
print(f'고유 아이템 type : {num_type}')

주문 정보 총량 : 177961
고유 아이템 code : 735
고유 아이템 desc : 735
고유 아이템 type : 27


### preprocess

In [38]:
# # date 기준으로 PO_RECEIPT 더하기

# dates = df_inv['YYYYMM'].unique()

# df_po = pd.DataFrame()
# columns = ['YYYYMM']

# for i, date in enumerate(dates):
#     tmp_date = df_inv.loc[df_inv['YYYYMM'] == date].reset_index(drop=True)

#     df_po.loc[i, columns] = tmp_date.loc[0, columns]
#     df_po.loc[i, 'PO_TOTAL'] = (tmp_date['PO_RECEIPT']).sum() 

In [8]:
# df_po.columns = ['ds', 'y']

In [14]:
# from datetime import datetime
# df_po['ds'] = df_po['ds'].apply(lambda _ : datetime.strptime(_,'%Y%m'))
# df_po['ds']